This notebook was created and run on Databricks, and will need additional setups to be run on Colab and Jupyter Notebook.

**Import Libraries**

In [3]:
!pip install emojis
!pip install nltk
import emojis
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import  *
from pyspark.sql import DataFrame
from collections import Counter
from typing import Iterable
import matplotlib.pyplot as plt
import seaborn as sns

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Package wordnet is already up-to-date!

**Import Dataset**

In [5]:
data = spark.read.parquet("/FileStore/tables/VenmoSample_snappy-e020d.parquet")
data.cache()   # cache to speed up following operations
display(data)

user1,user2,transaction_type,datetime,description,is_business,story_id
1218774,1528945,payment,2015-11-27T10:48:19.000+0000,Uber,false,5657c473cd03c9af22cff874
5109483,4782303,payment,2015-06-17T11:37:04.000+0000,Costco,false,5580f9702b64f70ab0114e94
4322148,3392963,payment,2015-06-19T07:05:31.000+0000,Sweaty balls,false,55835ccb1a624b14ac62cef4
469894,1333620,charge,2016-06-03T23:34:13.000+0000,🎥,false,5751b185cd03c9af224c0d17
2960727,3442373,payment,2016-05-29T23:23:42.000+0000,⚡,false,574b178ecd03c9af22cf67f4
3977544,2709470,payment,2016-09-29T22:12:07.000+0000,Chipotlaid,false,57ed2f4723e064eac0879304
3766386,4209061,payment,2016-05-20T10:31:15.000+0000,kitchen counter,false,573e8503cd03c9af2200360e
730075,804466,payment,2016-05-26T04:46:45.000+0000,Food,false,57461d46cd03c9af227dade4
5221751,4993533,payment,2016-07-14T22:53:49.000+0000,Zaxby,false,5787b58d23e064eac0126cfa
6843582,7308338,payment,2016-08-31T10:32:46.000+0000,Fan sucks,false,57c64fdf23e064eac0e88f69


In [6]:
# increase partition to (hopefully) speed up
data = data.repartition(200)
data.rdd.getNumPartitions()

Out[3]: 200

### **Text Analytics**

**Q0 [5 pts]​: Your first task is to open your Venmo app, find 10 words that are not already in the dictionary and add them to it. Make sure you don’t add to the dictionary a duplicate word by hitting Control+F before adding your word.**

The 10 new words we added are: 
1. tomato
2. popeye
3. carrot
4. hulu
5. hbo
6. mushroom
7. ups
8. fedex
9. wholefoods
10. italian

In [10]:
text_dic = spark.read.format("csv") \
  .option("header", "true")  \
  .option("sep", ",") \
  .load("/FileStore/tables/Venmo_Word_Classification_Dictonary_BAX_243___Word_Dict-5dd43.csv")
display(text_dic)

People,Food,Event,Activity,Travel,Transportation,Utility,Cash,Illegal/Sarcasm,_c9
friend,food,birthday,ball,beach,lyft,bill,atm,addiction,null
friendship,bbq,christmas,boat,place,uber,cable,bank,drug,null
baby,bean,happy,bar,la,cab,fee,cash,wangs,null
boy,latte,bday,book,world,bus,electric,money,weed,null
girl,breakfast,wedding,club,hotel,car,electricity,buck,anal,null
help,brunch,xmas,card,trip,gas,internet,wallet,bj,null
like,burger,holiday,dance,vega,taxi,rent,monies,blowjob,null
love,burrito,hbd,football,tahoe,ride,wifi,tip,boob,null
mom,cake,halloween,fun,nyc,rental,utility,dollar,booty,null
save,cheese,thanksgiving,game,dc,train,tax,payback,blow,null


In [11]:
emoji_dic = spark.read.format("csv") \
  .option("header", "true")  \
  .option("sep", ",") \
  .load("/FileStore/tables/Venmo_Emoji_Classification_Dictionary.csv")
display(emoji_dic)

Event,Travel,Food,Activity,Transportation,People,Utility
🇦🇺,🏔,🍇,👾,🚄,😀,⚡
🇫🇷,⛰,🍈,🕴,🚅,😃,💡
🎂,🌋,🍉,🎪,🚆,😄,🔌
🛍,🗻,🍊,🎭,🚇,😁,📺
🇨🇦,🏕,🍋,🎨,🚈,😆,🔌
🇧🇷,🏖,🍌,🎰,🚉,😅,⚡
🐉,🏜,🍍,🚣,🚊,🤣,💡
🎅,🏝,🍎,🛀,🚝,😂,💸
🇲🇽,🏞,🍏,🎗,🚞,🙂,💦
🇨🇳,🏟,🍐,🎟,🚋,🙃-,💧


**Q1 [2 pts]​: Use the text dictionary and the emoji dictionary to classify Venmo’s transactions in your sample dataset.**

Process: replace emoji with text -> remove punctuations -> tokenize -> remove stopwords -> lemmatize -> map to topic dictionary

In [14]:
people = text_dic.select('People ').rdd.flatMap(lambda x: x).collect()
food = text_dic.select('Food ').rdd.flatMap(lambda x: x).collect()
event = text_dic.select('Event ').rdd.flatMap(lambda x: x).collect()
activity = text_dic.select('Activity').rdd.flatMap(lambda x: x).collect()
travel = text_dic.select('Travel ').rdd.flatMap(lambda x: x).collect()
transportation = text_dic.select('Transportation ').rdd.flatMap(lambda x: x).collect()
utility = text_dic.select('Utility ').rdd.flatMap(lambda x: x).collect()
cash = text_dic.select('Cash').rdd.flatMap(lambda x: x).collect()
illegal = text_dic.select('Illegal/Sarcasm ').rdd.flatMap(lambda x: x).collect()

In [15]:
people_emoji = emoji_dic.select('People').rdd.flatMap(lambda x: x).collect()
food_emoji = emoji_dic.select('Food').rdd.flatMap(lambda x: x).collect()
event_emoji = emoji_dic.select('Event').rdd.flatMap(lambda x: x).collect()
activity_emoji = emoji_dic.select('Activity').rdd.flatMap(lambda x: x).collect()
travel_emoji = emoji_dic.select('Travel').rdd.flatMap(lambda x: x).collect()
transportation_emoji = emoji_dic.select('Transportation').rdd.flatMap(lambda x: x).collect()
utility_emoji = emoji_dic.select('Utility').rdd.flatMap(lambda x: x).collect()

In [16]:
# use the emojis package instead of the provided dictionary, not sure if it's allowed, need to check with professor
@udf
def convert_emojis(text):
    text = " ".join(emojis.decode(text).replace(":", " ").replace("_", "").split())
    return text

In [17]:
@udf
def rm_punctuation(text):
  return text.translate(str.maketrans("","", string.punctuation))

In [18]:
def lemmatize(text):
    lemmatized_array = []
    lemmatizer = WordNetLemmatizer()
    for word in text:
      lemmatized_array.append(lemmatizer.lemmatize(word))
    return lemmatized_array
lemmatize_udf = udf(lemmatize, ArrayType(StringType()))

In [19]:
data = data.withColumn("replace_emoji", convert_emojis(col("description")))
data = data.withColumn("rm_punctuation", rm_punctuation(col("replace_emoji")))
tokenizer = Tokenizer(inputCol="rm_punctuation", outputCol="tokenized")
data = tokenizer.transform(data)
remover = StopWordsRemover(inputCol="tokenized", outputCol="rm_stopwords")
data = remover.transform(data)
data = data.withColumn("text_preprocessed", lemmatize_udf(col("rm_stopwords")))

columns_to_drop = ['replace_emoji', 'tokenized', 'rm_stopwords', "rm_punctuation"]
data = data.drop(*columns_to_drop)
display(data)

user1,user2,transaction_type,datetime,description,is_business,story_id,text_preprocessed
1293545,3555063,payment,2015-06-28T23:51:11.000+0000,🍣🐠🍻,false,559025ff1a3b580f92a5025e,"List(sushi, tropicalfish, beer)"
6785327,5649602,payment,2016-06-14T08:11:20.000+0000,☎ 😘,false,575f59b8cd03c9af22380385,"List(☎, kissingheart)"
2514131,1531398,payment,2016-05-08T07:17:53.000+0000,NyQuil x2,false,572e85b1cd03c9af22eb2b82,"List(nyquil, x2)"
1003716,703145,payment,2015-02-24T02:59:58.000+0000,Food!,false,54eb78aecd03c9af22d7f0c0,List(food)
483288,314523,charge,2015-04-05T23:28:48.000+0000,BD,false,552162c0ca81793bbe80320b,List(bd)
3292421,3219776,payment,2015-02-05T06:08:34.000+0000,👽,false,54d2986291bd05aa933ea834,List(alien)
2014167,4633184,charge,2015-09-28T21:44:19.000+0000,PGW,false,56095243cd03c9af220240d3,List(pgw)
1151815,1474712,charge,2015-03-31T08:31:02.000+0000,for rent,false,5519f8d75d6cc87743b26b1c,List(rent)
481776,324017,charge,2016-02-08T06:29:19.000+0000,Supabowl groceries,false,56b7c53fcd03c9af220e7730,"List(supabowl, grocery)"
288481,571272,charge,2015-10-06T05:02:58.000+0000,🍴🚿,false,5612f392cd03c9af22819aaf,"List(forkandknife, shower)"


In [20]:
# use array to store the categories for each description, then get the most common one
@udf
def classification(text):
  categories = []
  for word in text:
    if word in people:
      categories.append('People')
    if word in food:
      categories.append('Food')
    if word in event:
      categories.append('Event')
    if word in activity:
      categories.append('Activity')
    if word in travel:
      categories.append('Travel')
    if word in transportation:
      categories.append('Transportation')
    if word in utility:
      categories.append('Utility')
    if word in cash:
      categories.append('Cash')
    if word in illegal:
      categories.append('Illegal')
  if not categories:
    return "Not Classified"
  else:
    return Counter(categories).most_common(1)[0][0]

In [21]:
data = data.withColumn("classification", classification(col("text_preprocessed")))
display(data)

user1,user2,transaction_type,datetime,description,is_business,story_id,text_preprocessed,classification
1293545,3555063,payment,2015-06-28T23:51:11.000+0000,🍣🐠🍻,false,559025ff1a3b580f92a5025e,"List(sushi, tropicalfish, beer)",Food
6785327,5649602,payment,2016-06-14T08:11:20.000+0000,☎ 😘,false,575f59b8cd03c9af22380385,"List(☎, kissingheart)",Not Classified
2514131,1531398,payment,2016-05-08T07:17:53.000+0000,NyQuil x2,false,572e85b1cd03c9af22eb2b82,"List(nyquil, x2)",Not Classified
1003716,703145,payment,2015-02-24T02:59:58.000+0000,Food!,false,54eb78aecd03c9af22d7f0c0,List(food),Food
483288,314523,charge,2015-04-05T23:28:48.000+0000,BD,false,552162c0ca81793bbe80320b,List(bd),Not Classified
3292421,3219776,payment,2015-02-05T06:08:34.000+0000,👽,false,54d2986291bd05aa933ea834,List(alien),Not Classified
2014167,4633184,charge,2015-09-28T21:44:19.000+0000,PGW,false,56095243cd03c9af220240d3,List(pgw),Not Classified
1151815,1474712,charge,2015-03-31T08:31:02.000+0000,for rent,false,5519f8d75d6cc87743b26b1c,List(rent),Utility
481776,324017,charge,2016-02-08T06:29:19.000+0000,Supabowl groceries,false,56b7c53fcd03c9af220e7730,"List(supabowl, grocery)",Food
288481,571272,charge,2015-10-06T05:02:58.000+0000,🍴🚿,false,5612f392cd03c9af22819aaf,"List(forkandknife, shower)",Not Classified


**Q2 [3 pts]​: What is the percent of emoji only transactions? Which are the top 5 most popular emoji? Which are the top three most popular emoji categories?**

In [23]:
@udf
def emoji_only(text):
  if len(text) == emojis.count(text):
    return 1
  return 0

In [24]:
data = data.withColumn("emoji_only", emoji_only(col("description")))
display(data)

user1,user2,transaction_type,datetime,description,is_business,story_id,text_preprocessed,classification,emoji_only
1293545,3555063,payment,2015-06-28T23:51:11.000+0000,🍣🐠🍻,false,559025ff1a3b580f92a5025e,"List(sushi, tropicalfish, beer)",Food,1
6785327,5649602,payment,2016-06-14T08:11:20.000+0000,☎ 😘,false,575f59b8cd03c9af22380385,"List(☎, kissingheart)",Not Classified,0
2514131,1531398,payment,2016-05-08T07:17:53.000+0000,NyQuil x2,false,572e85b1cd03c9af22eb2b82,"List(nyquil, x2)",Not Classified,0
1003716,703145,payment,2015-02-24T02:59:58.000+0000,Food!,false,54eb78aecd03c9af22d7f0c0,List(food),Food,0
483288,314523,charge,2015-04-05T23:28:48.000+0000,BD,false,552162c0ca81793bbe80320b,List(bd),Not Classified,0
3292421,3219776,payment,2015-02-05T06:08:34.000+0000,👽,false,54d2986291bd05aa933ea834,List(alien),Not Classified,1
2014167,4633184,charge,2015-09-28T21:44:19.000+0000,PGW,false,56095243cd03c9af220240d3,List(pgw),Not Classified,0
1151815,1474712,charge,2015-03-31T08:31:02.000+0000,for rent,false,5519f8d75d6cc87743b26b1c,List(rent),Utility,0
481776,324017,charge,2016-02-08T06:29:19.000+0000,Supabowl groceries,false,56b7c53fcd03c9af220e7730,"List(supabowl, grocery)",Food,0
288481,571272,charge,2015-10-06T05:02:58.000+0000,🍴🚿,false,5612f392cd03c9af22819aaf,"List(forkandknife, shower)",Not Classified,1


In [25]:
# 21% of transactions are emoji only
percent_emoji_only = data.select(sum("emoji_only")/data.count()).show()
percent_emoji_only

+---------------------------+
(sum(emoji_only) / 7113137)|
+---------------------------+
 0.2135835426760373|
+---------------------------+

In [26]:
def get_emoji(text):
  if emojis.get(text):
    return list(emojis.get(text))
get_emoji_udf = udf(get_emoji, ArrayType(StringType()))

In [27]:
data = data.withColumn("emojis", get_emoji_udf(col("description")))
display(data)

user1,user2,transaction_type,datetime,description,is_business,story_id,text_preprocessed,classification,emoji_only,emojis
1293545,3555063,payment,2015-06-28T23:51:11.000+0000,🍣🐠🍻,false,559025ff1a3b580f92a5025e,"List(sushi, tropicalfish, beer)",Food,1,"List(🐠, 🍻, 🍣)"
6785327,5649602,payment,2016-06-14T08:11:20.000+0000,☎ 😘,false,575f59b8cd03c9af22380385,"List(☎, kissingheart)",Not Classified,0,List(😘)
2514131,1531398,payment,2016-05-08T07:17:53.000+0000,NyQuil x2,false,572e85b1cd03c9af22eb2b82,"List(nyquil, x2)",Not Classified,0,null
1003716,703145,payment,2015-02-24T02:59:58.000+0000,Food!,false,54eb78aecd03c9af22d7f0c0,List(food),Food,0,null
483288,314523,charge,2015-04-05T23:28:48.000+0000,BD,false,552162c0ca81793bbe80320b,List(bd),Not Classified,0,null
3292421,3219776,payment,2015-02-05T06:08:34.000+0000,👽,false,54d2986291bd05aa933ea834,List(alien),Not Classified,1,List(👽)
2014167,4633184,charge,2015-09-28T21:44:19.000+0000,PGW,false,56095243cd03c9af220240d3,List(pgw),Not Classified,0,null
1151815,1474712,charge,2015-03-31T08:31:02.000+0000,for rent,false,5519f8d75d6cc87743b26b1c,List(rent),Utility,0,null
481776,324017,charge,2016-02-08T06:29:19.000+0000,Supabowl groceries,false,56b7c53fcd03c9af220e7730,"List(supabowl, grocery)",Food,0,null
288481,571272,charge,2015-10-06T05:02:58.000+0000,🍴🚿,false,5612f392cd03c9af22819aaf,"List(forkandknife, shower)",Not Classified,1,"List(🍴, 🚿)"


In [28]:
bag_of_emojis = data.filter(col("emojis").isNotNull()).select("emojis").rdd.flatMap(lambda x: x).collect()
bag_of_emojis_flatten = []
for item in bag_of_emojis:
  for emoji in item:
    bag_of_emojis_flatten.append(emoji)
bag_of_emojis_flatten

Out[19]: ['✨',
 '🍷',
 '🍷',
 '🍷',
 '🍷',
 '👙',
 '💃',
 '💄',
 '💃',
 '🐝',
 '🐝',
 '👚',
 '🏋',
 '🍽',
 '🍷',
 '🍸',
 '🍸',
 '🏡',
 '⚡',
 '👙',
 '💁',
 '🌊',
 '🏈',
 '🐻',
 '🍺',
 '🚖',
 '💍',
 '💍',
 '❤️',
 '🎄',
 '♀️',
 '🚴',
 '🍽',
 '💃',
 '🎉',
 '😚',
 '✏️',
 '🇰🇷',
 '🇰🇷',
 '🍝',
 '🍴',
 '🌨',
 '😉',
 '⛽',
 '💸',
 '💻',
 '🎂',
 '❤️',
 '🛫',
 '🌱',
 '🌿',
 '🍃',
 '🎉',
 '📺',
 '💻',
 '🍬',
 '🍪',
 '🌱',
 '🌿',
 '🌳',
 '🌼',
 '🌷',
 '🌻',
 '⛰',
 '🚗',
 '⛽',
 '⛳',
 '🏌',
 '💯',
 '🏄',
 '🍳',
 '☕',
 '🙋',
 '🚙',
 '🐳',
 '🐳',
 '🍻',
 '🍺',
 '🍔',
 '🍵',
 '🐾',
 '😋',
 '🍻',
 '🍳',
 '🙌',
 '👸',
 '🍸',
 '🎉',
 '🙌',
 '💕',
 '🍺',
 '🍻',
 '💌',
 '💌',
 '🍗',
 '🍰',
 '💰',
 '🐶',
 '🍏',
 '🍏',
 '🍴',
 '🎎',
 '💡',
 '🏸',
 '🔌',
 '📺',
 '💡',
 '🍼',
 '🍌',
 '🏂',
 '💸',
 '🏠',
 '😄',
 '🇲🇽',
 '🎶',
 '⛳',
 '👊',
 '🌭',
 '🌭',
 '🌭',
 '🔵',
 '🚙',
 '⛵',
 '🚖',
 '👉',
 '👵',
 '🍨',
 '👧',
 '💡',
 '🍗',
 '🍟',
 '🍔',
 '🌭',
 '🍜',
 '🍹',
 '🎓',
 '🍕',
 '🍕',
 '🍕',
 '🍕',
 '😭',
 '👷',
 '🍳',
 '🧀',
 '🌮',
 '🍴',
 '🍤',
 '🍇',
 '🍞',
 '🍝',
 '🍞',
 '🍤',
 '🍙',
 '🍖',
 '🍜',
 '🐣',
 '🐥',
 '💻',
 '🍗',
 '🍗',
 '🍗',
 '🍗',
 '🍗',
 '🍗',
 '🍗',
 '🍗',
 '🍗',
 '🍗',
 '🍗',
 '🍗',
 '🐙',
 '👹',
 '🐣',
 '💃',
 '💃',
 '📱',
 '💸',
 '🍕',
 '🚖',
 '💑',
 '💢',
 '👊',
 '🤘',
 '😕',
 '💰',
 '🏠',
 '💦',
 '🍻',
 '🍻',
 '🍔',
 '🙂',
 '😋',
 '🐮',
 '🎉',
 '👿',
 '🍛',
 '🌊',
 '🌊',
 '🌊',
 '🌃',
 '🎲',
 '🏨',
 '🍕',
 '🍺',
 '👌',
 '📩',
 '📨',
 '💝',
 '🌍',
 '🙌',
 '😎',
 '🔌',
 '💡',
 '💻',
 '🖥',
 '😊',
 '🌸',
 '😎',
 '⚽',
 '🇯🇵',
 '💅',
 '👢',
 '😝',
 '😝',
 '💃',
 '🌮',
 '❄️',
 '⚡',
 '🎆',
 '🍾',
 '🌨',
 '🏡',
 '🍻',
 '⚓',
 '✈️',
 '🐯',
 '💪',
 '💁',
 '🍕',
 '👜',
 '🏂',
 '🍉',
 '🍉',
 '🍉',
 '💰',
 '🍲',
 '🍲',
 '🍲',
 '🍲',
 '🍲',
 '🍲',
 '🍲',
 '🎉',
 '🚿',
 '🌯',
 '🍻',
 '🌮',
 '🎉',
 '🍾',
 '💍',
 '👎',
 '💪',
 '🙊',
 '🍕',
 '👍',
 '🏈',
 '🍻',
 '🌝',
 '🛍',
 '😘',
 '💸',
 '🏠',
 '💸',
 '🏠',
 '⛳',
 '🏌',
 '👧',
 '👍',
 '💙',
 '🚘',
 '💃',
 '👕',
 '👌',
 '🎳',
 '🎂',
 '🎉',
 '🌈',
 '😘',
 '👸',
 '❤️',
 '🍷',
 '📸',
 '🎗',
 '🏀',
 '🏆',
 '😡',
 '🏠',
 '🍕',
 '🙆',
 '👯',
 '🔌',
 '💵',
 '🦄',
 '🍆',
 '🎉',
 '📺',
 '💰',
 '🙆',
 '🚿',
 '🚽',
 '🛁',
 '🚀',
 '🚀',
 '🚀',
 '🚀',
 '🏆',
 '🐶',
 '💸',
 '👟',
 '🚌',
 '🍸',
 '🎉',
 '🏠',
 '😋',
 '🦁',
 '🏜',
 '🎠',
 '🎸',
 '🌠',
 '🎼',
 '🎡',
 '🎇',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '🎉',
 '😜',
 '💵',
 '🐘',
 '💺',
 '😎',
 '😋',
 '🍝',
 '👌',
 '🐛',
 '🐛',
 '💰',
 '💰',
 '👍',
 '🍟',
 '😜',
 '🍔',
 '🌻',
 '🌻',
 '👅',
 '📔',
 '➕',
 '🍟',
 '✂️',
 '🍔',
 '🍵',
 '🍜',
 '🍞',
 '💁',
 '💁',
 '💁',
 '❤️',
 '👸',
 '🏙',
 '⛄',
 '🦀',
 '👬',
 '☀️',
 '🌊',
 '🌽',
 '🍻',
 '👕',
 '🍽',
 '🍾',
 '🍹',
 '🍋',
 '👒',
 '👨',
 '🎈',
 '🍈',
 '🚒',
 '🔌',
 '💡',
 '🏡',
 '🚊',
 '🍄',
 '🍄',
 '🍄',
 '😄',
 '🐪',
 '🐄',
 '😊',
 '💦',
 '🌟',
 '💃',
 '😝',
 '🌱',
 '🌿',
 '☘',
 '🎋',
 '🎍',
 '🌳',
 '🍂',
 '🌾',
 '🍁',
 '🎄',
 '🌲',
 '🍃',
 '🌴',
 '🍟',
 '🏡',
 '🚰',
 '👉',
 '👌',
 '🤗',
 '🤗',
 '🤗',
 '🤗',
 '🤗',
 '🤗',
 '🤗',
 '💶',
 '👽',
 '🔵',
 '👹',
 '👺',
 '🏀',
 '😲',
 '🍺',
 '🍷',
 '🍾',
 '🏊',
 '🚣',
 '🍻',
 '🍵',
 '⚽',
 '😋',
 '🎁',
 '🎁',
 '🎁',
 '🎁',
 '🎁',
 '🎁',
 '🎁',
 '🎁',
 '🎁',
 '🎁',
 '🎁',
 '🎁',
 '🎁',
 '🍣',
 '🇯🇴',
 '🐼',
 '🐹',
 '🐸',
 '🐷',
 '🐘',
 '🐑',
 '🏥',
 '👍',
 '👍',
 '👍',
 '👍',
 '🇵🇰',
 '🍩',
 '🍷',
 '🍪',
 '👍',
 '🚘',
 '🚙',
 '🌭',
 '🙊',
 '🍌',
 '💕',
 '✨',
 '😉',
 '😉',
 '😉',
 '😉',
 '😉',
 '😉',
 '😉',
 '🍕',
 '🍹',
 '🍻',
 '😇',
 '💃',
 '🏁',
 '🙌',
 '🐼',
 '😤',
 '👅',
 '😘',
 '👄',
 '🍕',
 '🍋',
 '🍏',
 '🌶',
 '👌',
 '🐶',
 '📱',
 '📺',
 '😀',
 '😀',
 '🐔',
 '🤖',
 '🤖',
 '🤖',
 '🤖',
 '💳',
 '😒',
 '🎅',
 '💆',
 '🚨',
 '🍵',
 '💡',
 '🔌',
 '🍷',
 '🍷',
 '🍷',
 '🍷',
 '🍷',
 '🍷',
 '😅',
 '🏂',
 '👏',
 '🍺',
 '🏢',
 '🍸',
 '📺',
 '💻',
 '🛍',
 '🍕',
 '😉',
 '🍚',
 '🍚',
 '🍚',
 '🍚',
 '🍚',
 '🍔',
 '🍻',
 '😤',
 '🍭',
 '🍻',
 '💍',
 '🍣',
 '🏀',
 '🏀',
 '💥',
 '🏈',
 '🏈',
 '💻',
 '💸',
 '⚡',
 '⛽',
 '🌮',
 '💾',
 '💻',
 '⏳',
 '💁',
 '💅',
 '🎊',
 '👊',
 '👖',
 '🌯',
 '😋',
 '🍟',
 '🌮',
 '🆒',
 '🆒',
 '😮',
 '😃',
 '🌞',
 '🌞',
 '🌞',
 '😍',
 '😘',
 '🍛',
 '🍛',
 '🍛',
 '🍛',
 '🍕',
 '🎤',
 '🍝',
 '🍷',
 '🚛',
 '💪',
 '🙌',
 '😩',
 '👟',
 '😎',
 '🍻',
 '😍',
 '💵',
 '🔙',
 '💵',
 '🔙',
 '💵',
 '🔙',
 '💵',
 '🔙',
 '💵',
 '🔙',
 '😤',
 '🚜',
 '🚗',
 '🙌',
 '💄',
 '🐤',
 '💃',
 '💁',
 '🍴',
 '🍃',
 '💳',
 '🏈',
 '💊',
 '🍩',
 '☕',
 '👀',
 '👀',
 '👀',
 '👀',
 '👀',
 '👀',
 '🔌',
 '💡',
 '💸',
 '⚡',
 '🔌',
 '💡',
 '💸',
 '⚡',
 '🔌',
 '💡',
 '💸',
 '⚡',
 '🔌',
 '💡',
 '💸

In [29]:
# the top 5 most popular emoji: '💸', '🍕', '🍻', '🎉', '🍷'
Counter(bag_of_emojis_flatten).most_common(5)

Out[20]: [('💸', 111172), ('🍕', 107661), ('🍻', 105950), ('🎉', 73014), ('🍷', 70474)]

In [30]:
def get_emoji_category(emoji):
  if emoji in people_emoji:
    return 'People'
  elif emoji in food_emoji:
    return 'Food'
  elif emoji in event_emoji:
    return 'Event'
  elif emoji in activity_emoji:
    return 'Activity'
  elif emoji in travel_emoji:
    return 'Travel'
  elif emoji in transportation_emoji:
    return 'Transportation'
  elif emoji in utility_emoji:
    return 'Utility'
  else:
    return 'Not Classified'

In [31]:
# use dictionary to speed up the running time
emoji_category_dic = dict()
emoji_category_arr = []

for emoji in bag_of_emojis_flatten:
  if emoji not in emoji_category_dic:
    emoji_category_dic[emoji] = get_emoji_category(emoji)
    emoji_category_arr.append(emoji_category_dic[emoji])
  else:
    emoji_category_arr.append(emoji_category_dic[emoji])

emoji_category_arr = [x for x in emoji_category_arr if x != "Not Classified"]
emoji_category_arr

Out[22]: ['Food',
 'Food',
 'Food',
 'Food',
 'People',
 'People',
 'People',
 'People',
 'Activity',
 'Food',
 'Food',
 'Food',
 'Food',
 'Utility',
 'People',
 'People',
 'Activity',
 'Food',
 'Transportation',
 'People',
 'People',
 'Activity',
 'Food',
 'People',
 'Activity',
 'People',
 'Food',
 'Food',
 'People',
 'Transportation',
 'Event',
 'Transportation',
 'Event',
 'Activity',
 'Utility',
 'Food',
 'Food',
 'Event',
 'Travel',
 'Transportation',
 'Transportation',
 'Activity',
 'Activity',
 'Activity',
 'Food',
 'Food',
 'People',
 'Food',
 'Food',
 'Food',
 'Food',
 'People',
 'Food',
 'Food',
 'People',
 'People',
 'Food',
 'Activity',
 'People',
 'People',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Utility',
 'Activity',
 'Utility',
 'Utility',
 'Utility',
 'Food',
 'Food',
 'Activity',
 'People',
 'Event',
 'Activity',
 'People',
 'Food',
 'Food',
 'Food',
 'Transportation',
 'Transportation',
 'People',
 'People',
 'Food',
 'People',
 'Utility',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Event',
 'Food',
 'Food',
 'Food',
 'Food',
 'People',
 'People',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'People',
 'People',
 'People',
 'Utility',
 'Food',
 'Transportation',
 'People',
 'People',
 'People',
 'People',
 'Utility',
 'Food',
 'Food',
 'Food',
 'People',
 'People',
 'Activity',
 'People',
 'Food',
 'Travel',
 'Activity',
 'Travel',
 'Food',
 'Food',
 'People',
 'People',
 'People',
 'Utility',
 'Utility',
 'People',
 'People',
 'Activity',
 'People',
 'People',
 'People',
 'People',
 'Food',
 'Utility',
 'Food',
 'Food',
 'Transportation',
 'Event',
 'People',
 'Food',
 'People',
 'Activity',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Activity',
 'Food',
 'Food',
 'Food',
 'Activity',
 'Food',
 'People',
 'People',
 'Event',
 'Food',
 'People',
 'Activity',
 'Food',
 'Event',
 'People',
 'Activity',
 'Activity',
 'People',
 'People',
 'Transportation',
 'People',
 'People',
 'Activity',
 'Event',
 'Activity',
 'People',
 'People',
 'Food',
 'Activity',
 'Activity',
 'Activity',
 'People',
 'Food',
 'People',
 'People',
 'Utility',
 'Travel',
 'Food',
 'Activity',
 'Utility',
 'People',
 'Travel',
 'Travel',
 'Travel',
 'Travel',
 'Activity',
 'People',
 'Transportation',
 'Food',
 'Activity',
 'People',
 'Travel',
 'Travel',
 'Activity',
 'Travel',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'People',
 'Travel',
 'Transportation',
 'People',
 'People',
 'Food',
 'People',
 'People',
 'Food',
 'People',
 'Food',
 'People',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'People',
 'People',
 'People',
 'People',
 'Travel',
 'Event',
 'People',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Food',
 'Event',
 'Food',
 'Transportation',
 'Utility',
 'Utility',
 'Transportation',
 'Food',
 'Food',
 'Food',
 'People',
 'People',
 'Utility',
 'People',
 'People',
 'Event',
 'Event',
 'Event',
 'Food',
 'People',
 'People',
 'People',
 'People',
 'People',
 'People',
 'People',
 'People',
 'People',
 'Travel',
 'People',
 'People',
 'People',
 'Activity',
 'People',
 'Food',
 'Food',
 'Food',
 'Activity',
 'Activity',
 'Food',
 'Food',
 'Activity',
 'People',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Activity',
 'Food',
 'Travel',
 'People',
 'People',
 'People',
 'People',
 'Food',
 'Food',
 'Food',
 'People',
 'Transportation',
 'Food',
 'Food',
 'People',
 'People',
 'People',
 'People',

In [32]:
# the top three most popular emoji categories are Food, People, Activity
Counter(emoji_category_arr).most_common(3)

Out[23]: [('Food', 1214868), ('People', 750489), ('Activity', 320200)]

**Q3 [2 pts]​: For each user, create a variable to indicate their spending behavior profile. For example, if a user has made 10 transactions, where 5 of them are food and the other 5 are activity, then the user’s spending profile will be 50% food and 50% activity.**

In [34]:
data.createOrReplaceTempView("datatable")

In [35]:
%sql
-- merge user1 and user2
CREATE EXTERNAL TABLE union_table
USING HIVE
AS 
  SELECT user1 AS user, classification
  FROM datatable
  UNION ALL
  SELECT user2 AS user, classification
  FROM datatable

In [36]:
# assumping the denominator doesn't include transactions that can't be classified
spend_profile_df = sqlContext.sql(
  '''
  SELECT user, CONCAT_WS(", ", 
                    CASE WHEN count_activity != 0 THEN CONCAT(ROUND(count_activity * 100), "% Activity") ELSE NULL END, 
                    CASE WHEN count_people != 0 THEN CONCAT(ROUND(count_people * 100), "% People") ELSE NULL END, 
                    CASE WHEN count_event != 0 THEN CONCAT(ROUND(count_event * 100), "% Event") ELSE NULL END, 
                    CASE WHEN count_travel != 0 THEN CONCAT(ROUND(count_travel * 100), "% Travel") ELSE NULL END, 
                    CASE WHEN count_transportation != 0 THEN CONCAT(ROUND(count_transportation * 100), "% Transportation") ELSE NULL END, 
                    CASE WHEN count_food != 0 THEN CONCAT(ROUND(count_food * 100), "% Food") ELSE NULL END, 
                    CASE WHEN count_utility != 0 THEN CONCAT(ROUND(count_utility * 100), "% Utility") ELSE NULL END, 
                    CASE WHEN count_cash != 0 THEN CONCAT(ROUND(count_cash * 100), "% Cash") ELSE NULL END,
                    CASE WHEN count_illegal != 0 THEN CONCAT(ROUND(count_illegal * 100), "% Illegal") ELSE NULL END
              ) AS spent_profile
  FROM
  (
    SELECT user,
        SUM(CASE WHEN classification == "Activity" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_activity,
        SUM(CASE WHEN classification == "Food" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_food,
        SUM(CASE WHEN classification == "People" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_people,
        SUM(CASE WHEN classification == "Event" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_event,
        SUM(CASE WHEN classification == "Travel" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_travel,
        SUM(CASE WHEN classification == "Transportation" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_transportation,
        SUM(CASE WHEN classification == "Utility" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_utility,
        SUM(CASE WHEN classification == "Cash" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_cash,
        SUM(CASE WHEN classification == "Illegal" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_illegal
    FROM union_table
    GROUP BY user
  )
  '''
)
display(spend_profile_df)

user,spent_profile
8306814,"50.0% Activity, 50.0% Food"
4810394,"50.0% Activity, 50.0% Food"
861694,
384959,"13.0% Activity, 6.0% Travel, 6.0% Transportation, 31.0% Food, 25.0% Utility, 19.0% Illegal"
372733,"8.0% Transportation, 15.0% Food, 62.0% Utility, 15.0% Illegal"
3583078,"13.0% Activity, 13.0% Travel, 38.0% Food, 25.0% Utility, 13.0% Cash"
580990,
2319823,"25.0% Activity, 25.0% People, 25.0% Event, 25.0% Transportation"
7016521,100.0% Food
663893,100.0% Utility


In [37]:
# export the file to faster next time execution
# spend_profile_df.write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/spend_profile_df.csv")

**Q4 [3 pts]​: In the previous question, you got a static spending profile. However, life and social networks are evolving over time. Therefore, let’s explore how a user’s spending profile is evolving over her lifetime in Venmo. First of all, you need to analyze a user’s transactions in monthly intervals, starting from 0 (indicating ​their first transaction only​) up to 12.**

In [39]:
# merge user1 & user2
union_table_with_date_df = sqlContext.sql(
  '''
  SELECT user1 AS user, classification, datetime
  FROM datatable
  UNION ALL
  SELECT user2 AS user, classification, datetime
  FROM datatable
  '''
)

In [40]:
# save to storage
# union_table_with_date_df.write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/union_table_with_date_df.csv")
# display(union_table_with_date_df)

user,classification,datetime
2514280,Not Classified,2015-10-31T03:46:49.000+0000
2371358,Not Classified,2016-06-01T06:24:59.000+0000
1409159,Utility,2016-07-01T01:35:57.000+0000
7075328,Not Classified,2016-03-19T08:25:43.000+0000
4093977,Not Classified,2016-02-02T10:41:52.000+0000
3466836,Food,2015-08-22T05:56:39.000+0000
1521281,Activity,2015-09-25T04:14:02.000+0000
4344715,Transportation,2016-02-17T15:41:52.000+0000
800698,Not Classified,2014-10-19T00:54:05.000+0000
3725355,Not Classified,2016-07-14T05:09:41.000+0000


In [41]:
# read from storage
union_table_with_date_df = spark.read.format('csv').options(header='true', inferSchema='true')\
    .load('dbfs:/FileStore/union_table_with_date_df.csv')
union_table_with_date_df.createOrReplaceTempView("uniontable")

In [42]:
# find the first 12 life points
# assumption: 30 days a month
union_table_label_lifepoints = sqlContext.sql(
  '''
  SELECT user, classification, life_point
  FROM
  (
    SELECT user, classification,
      CASE WHEN datetime = first_day THEN 0
           WHEN DATEDIFF(datetime, first_day) <= 30 THEN 1
           WHEN DATEDIFF(datetime, first_day) <= 60 THEN 2
           WHEN DATEDIFF(datetime, first_day) <= 90 THEN 3
           WHEN DATEDIFF(datetime, first_day) <= 120 THEN 4
           WHEN DATEDIFF(datetime, first_day) <= 150 THEN 5
           WHEN DATEDIFF(datetime, first_day) <= 180 THEN 6
           WHEN DATEDIFF(datetime, first_day) <= 210 THEN 7
           WHEN DATEDIFF(datetime, first_day) <= 240 THEN 8
           WHEN DATEDIFF(datetime, first_day) <= 270 THEN 9
           WHEN DATEDIFF(datetime, first_day) <= 300 THEN 10
           WHEN DATEDIFF(datetime, first_day) <= 330 THEN 11
           WHEN DATEDIFF(datetime, first_day) <= 360 THEN 12
      END AS life_point
    FROM 
    (
      SELECT *, MIN(datetime) OVER (PARTITION BY user) AS first_day
      FROM uniontable
    )
  )
  WHERE life_point <= 12
  '''
)
display(union_table_label_lifepoints)

user,classification,life_point
2866,Not Classified,0
3918,Food,1
3918,Food,0
4935,Not Classified,0
5300,Activity,4
5300,Food,6
5300,Not Classified,6
5300,Not Classified,0
6620,Travel,0
20735,Food,0


In [43]:
# save to local to speed up later operations
# dbutils.fs.rm('dbfs:/FileStore/union_table_label_lifepoints.csv', True) 
# union_table_label_lifepoints.write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/union_table_label_lifepoints.csv")

In [44]:
# read from storage
union_table_qualified_subset = spark.read.format('csv').options(header='true', inferSchema='true')\
    .load('dbfs:/FileStore/union_table_label_lifepoints.csv')
union_table_label_lifepoints.createOrReplaceTempView("union_table_subset")

In [45]:
%sql
--  count classifications grouping by user, life_point
--  assumping the denominator doesn't include transactions that can't be classified
DROP TABLE IF EXISTS classfication_by_time;

CREATE TABLE classfication_by_time
USING HIVE
AS 
  SELECT user, life_point,
    SUM(CASE WHEN classification == "Activity" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_activity,
    SUM(CASE WHEN classification == "Food" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_food,
    SUM(CASE WHEN classification == "People" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_people,
    SUM(CASE WHEN classification == "Event" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_event,
    SUM(CASE WHEN classification == "Travel" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_travel,
    SUM(CASE WHEN classification == "Transportation" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_transportation,
    SUM(CASE WHEN classification == "Utility" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_utility,
    SUM(CASE WHEN classification == "Cash" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_cash,
    SUM(CASE WHEN classification == "Illegal" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_illegal
  FROM union_table_subset
  GROUP BY user, life_point;
    
SELECT *
FROM classfication_by_time

user,life_point,count_activity,count_food,count_people,count_event,count_travel,count_transportation,count_utility,count_cash,count_illegal
2866,0,null,null,null,null,null,null,null,null,null
3918,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3918,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4935,0,null,null,null,null,null,null,null,null,null
5300,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5300,6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5300,0,null,null,null,null,null,null,null,null,null
6620,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
20735,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28170,0,null,null,null,null,null,null,null,null,null


In [46]:
# create profile by user, life_point
dynamic_profile_df = sqlContext.sql(
  '''
  SELECT user, life_point,
             CONCAT_WS(", ", 
                    CASE WHEN count_activity != 0 THEN CONCAT(ROUND(count_activity * 100), "% Activity") ELSE NULL END, 
                    CASE WHEN count_people != 0 THEN CONCAT(ROUND(count_people * 100), "% People") ELSE NULL END, 
                    CASE WHEN count_event != 0 THEN CONCAT(ROUND(count_event * 100), "% Event") ELSE NULL END, 
                    CASE WHEN count_travel != 0 THEN CONCAT(ROUND(count_travel * 100), "% Travel") ELSE NULL END, 
                    CASE WHEN count_transportation != 0 THEN CONCAT(ROUND(count_transportation * 100), "% Transportation") ELSE NULL END, 
                    CASE WHEN count_food != 0 THEN CONCAT(ROUND(count_food * 100), "% Food") ELSE NULL END, 
                    CASE WHEN count_utility != 0 THEN CONCAT(ROUND(count_utility * 100), "% Utility") ELSE NULL END, 
                    CASE WHEN count_cash != 0 THEN CONCAT(ROUND(count_cash * 100), "% Cash") ELSE NULL END,
                    CASE WHEN count_illegal != 0 THEN CONCAT(ROUND(count_illegal * 100), "% Illegal") ELSE NULL END
              ) AS spent_profile
  FROM classfication_by_time
  ORDER BY user, life_point
  '''
)
display(dynamic_profile_df)

user,life_point,spent_profile
2,0,
3,0,
3,1,100.0% Event
3,3,100.0% People
3,4,"50.0% Food, 50.0% Utility"
3,5,
4,0,100.0% Food
4,1,"50.0% Transportation, 50.0% Food"
4,2,100.0% Illegal
6,0,


In [47]:
# reshape dataframe from long to wide
dynamic_profile_df = dynamic_profile_df\
    .groupby(dynamic_profile_df.user)\
    .pivot("life_point")\
    .agg(first("spent_profile"))
display(dynamic_profile_df)  

user,1,2,3,4,5,6,7,8,9,10,11,12
2866,,null,null,null,null,null,null,null,null,null,null,null
3918,100.0% Food,null,null,null,null,null,null,null,null,null,null,null
4935,,null,null,null,null,null,null,null,null,null,null,null
5300,,100.0% Activity,100.0% Food,null,null,null,null,null,null,null,null,null
109050,100.0% People,100.0% People,null,null,null,null,null,null,null,null,null,null
109068,100.0% People,,null,null,null,null,null,null,null,null,null,null
109622,,,null,null,null,null,null,null,null,null,null,null
109909,,null,null,null,null,null,null,null,null,null,null,null
110081,100.0% Transportation,null,null,null,null,null,null,null,null,null,null,null
112020,100.0% Transportation,,100.0% Event,,100.0% Transportation,null,null,null,null,null,null,null


In [48]:
# save to local to speed up later operations
# dbutils.fs.rm('dbfs:/FileStore/dynamic_profile_df.csv', True) 
# dynamic_profile_df.write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/dynamic_profile_df.csv")

**for each time point, you need to compute the average and standard deviation of each spending category across all users**

In [50]:
# function to reshape df from wide to long
def melt(
        df: DataFrame, 
        id_vars: Iterable[str], value_vars: Iterable[str], 
        var_name: str="variable", value_name: str="value") -> DataFrame:
    """Convert :class:`DataFrame` from wide to long format."""

    # Create array<struct<variable: str, value: ...>>
    _vars_and_vals = array(*(
        struct(lit(c).alias(var_name), col(c).alias(value_name)) 
        for c in value_vars))

    # Add to the DataFrame and explode
    _tmp = df.withColumn("_vars_and_vals", explode(_vars_and_vals))

    cols = id_vars + [
            col("_vars_and_vals")[x].alias(x) for x in [var_name, value_name]]
    return _tmp.select(*cols)

In [51]:
# calculate the average
avg_by_time_df  = sqlContext.sql(
  '''
  SELECT life_point,
      AVG(count_activity) AS avg_activity,
      AVG(count_food) AS avg_food,
      AVG(count_people) AS avg_people,
      AVG(count_event) AS avg_event,
      AVG(count_travel) AS avg_travel,
      AVG(count_transportation) AS avg_transportation,
      AVG(count_utility) AS avg_utility,
      AVG(count_cash) AS avg_cash,
      AVG(count_illegal) AS avg_illegal
  FROM classfication_by_time
  GROUP BY life_point
  '''
)
avg_by_time_df = melt(avg_by_time_df, id_vars=['life_point'], value_vars=['avg_activity', 'avg_food', 'avg_people', 
                                                      'avg_event', 'avg_travel', 'avg_transportation', 'avg_utility', 
                                                                                  'avg_cash', 'avg_illegal'])
display(avg_by_time_df)

life_point,variable,value
12,avg_activity,0.1674840774142881
12,avg_food,0.36000181052123814
12,avg_people,0.07511122199186143
12,avg_event,0.022306352944997408
12,avg_travel,0.02713804414221233
12,avg_transportation,0.07641729391573683
12,avg_utility,0.17521136113810631
12,avg_cash,0.013220791611244691
12,avg_illegal,0.08310904632031482
1,avg_activity,0.17529532697183053


In [52]:
# calculate the std
std_by_time_df  = sqlContext.sql(
  '''
  SELECT life_point,
      STD(count_activity) AS std_activity,
      STD(count_food) AS std_food,
      STD(count_people) AS std_people,
      STD(count_event) AS std_event,
      STD(count_travel) AS std_travel,
      STD(count_transportation) AS std_transportation,
      STD(count_utility) AS std_utility,
      STD(count_cash) AS std_cash,
      STD(count_illegal) AS std_illegal
  FROM classfication_by_time
  GROUP BY life_point
  '''
)
std_by_time_df = melt(std_by_time_df, id_vars=['life_point'], value_vars=['std_activity', 'std_food', 'std_people', 
                                                      'std_event', 'std_travel', 'std_transportation', 'std_utility', 
                                                                                  'std_cash', 'std_illegal'])
display(std_by_time_df)

life_point,variable,value
12,std_activity,0.35986940422475394
12,std_food,0.4616661876278923
12,std_people,0.25316348312825365
12,std_event,0.14219794381486187
12,std_travel,0.15604884991041545
12,std_transportation,0.25418670567355495
12,std_utility,0.3667294628601329
12,std_cash,0.10934560899217934
12,std_illegal,0.26482525771370796
1,std_activity,0.36983208631519093


In [53]:
# convert to pandas dataframe for plotting
avg_by_time_pd = avg_by_time_df.toPandas()
std_by_time_pd = std_by_time_df.toPandas()

/databricks/spark/python/pyspark/sql/types.py:1636: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 return pa.schema(fields)

In [54]:
f, axes = plt.subplots(3, 3, figsize=(15, 10))
sns.despine(left=True)

variables  = ['activity', 'people', 'travel', 'transportation', 'food',  'illegal', 'cash', 'utility', 'event']
for idx, var in enumerate(variables):
  x = []
  lower_bound = []
  upper_bound = []
  avg_variable = 'avg_' + var
  std_variable = 'std_' + var
  for point in range(1, 13):
    avg  = avg_by_time_pd[(avg_by_time_pd.variable == avg_variable) & (avg_by_time_pd.life_point == point)].values[0][2]
    std = std_by_time_pd[(std_by_time_pd.variable == std_variable) & (std_by_time_pd.life_point == point)].values[0][2]
    x.append(avg)
    lower_bound.append(avg - 2 * std)
    upper_bound.append(avg + 2 * std)
  y = list(range(1, 13))
  
  # plot the confidence interval
  axes[idx // 3, idx % 3].set_xlim(-0.75,1)
  axes[idx // 3, idx % 3].fill_betweenx(y, lower_bound, upper_bound)
  # plot the average
  sns.scatterplot(x, y, ax=axes[idx // 3, idx % 3])
  axes[idx // 3, idx % 3].set_title(avg_variable)
  axes[idx // 3, idx % 3].set_ylabel("lifetime in months")

**Does the spending profile of the average customer stabilize after some point in time?**

Most of the average spendings stabilize after the first life point.